In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import subprocess
import traceback
from time import sleep
import requests

from anisotropy import run_SEPevent

import sys
import logging
filehandler = logging.FileHandler(filename="wind_events_test.log", encoding="utf-8")
streamhandler = logging.StreamHandler(sys.stdout)
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s', 
                    datefmt='%m/%d/%Y %H:%M:%S', handlers=[filehandler, streamhandler], force=True)

import warnings
warnings.filterwarnings("ignore", "WARNING: background mean is nan!", category=UserWarning, module="pyonset")

# def check_for_gaps(data, start, end, max_gap_mins=10):
#     flux_finite = data.dropna()
#     for i in range(len(flux_finite[start:end]) - 1):
#         if flux_finite.index[i + 1] - flux_finite.index[i] > pd.Timedelta(minutes=max_gap_mins):
#             return True
#     return False

year = 2012

data_path = f"{os.getcwd()}{os.sep}data"
csv_path = f"{os.getcwd()}{os.sep}wind_events_{year}.csv"
plot_dir = f"{os.getcwd()}{os.sep}/plots/{year}"
os.makedirs(plot_dir, exist_ok=True)

d = 0   # day counter
j = 0   # retry counter
MAX_RETRIES = 5

while d < 365:
    try:
        onset_found = False
        resample = "5min"
        resample_int = int(resample.removesuffix("min"))
        window_len = 1
        onsets = []
        peaks = []

        date = pd.to_datetime(f"{year}-01-01") + pd.Timedelta(days=d)
        next_date = date + pd.Timedelta(days=1)
        logging.info(f"Analyzing flux for {date}")

        start = date - pd.Timedelta(days=1)
        end = date + pd.Timedelta(days=2)

        # d = start.strftime("%Y%m%d")
        # while d != end:
        #     rc = subprocess.call(f'wget -r -nv --show-progress --tries=10 -nc -nH -np -nd -P "data" -A "*_{d}_*.cdf" "https://cdaweb.gsfc.nasa.gov/pub/data/wind/3dp/3dp_sfpd/{year}/"'.split(" "))
        
        event = run_SEPevent(data_path, spacecraft_instrument="Wind 3DP", starttime=start, endtime=end, 
                            species="e", channels=3, averaging=resample)         # wget data from CDAWeb if SSL Berkeley is down
        
        # Messy workaround.
        data = pd.Series(event.I_data[:,3], index=event.I_times).ffill(limit=2)
        event.wind_peak_removal(n_lim=2)   # Wind has some high flux peaks lasting for a couple of minutes. Remove these
        data = pd.Series(event.I_data[:,3], index=event.I_times).ffill(limit=2) # do this twice since first removal doesnt work if preceding value is missing -> fill missing values with previous values
        
        # 4-sigma method (Krucker et al. 1999, modified)        # TODO: 2.12. 3-sigma and 15 min averaging instead
        i = 0
        while i < len(data[date:next_date] - 1):
            # 1: calculate window mean and std
            window_end = date + pd.Timedelta(minutes=resample_int*i) # first window ends at the end of the 1st day
            window_start = window_end - pd.Timedelta(hours=window_len)
            # if check_for_gaps(data, window_start, window_end, max_gap_mins=10): 
            #     if not data_gaps_present:
            #         logging.info("Data gaps present, onset not determined")
            #         data_gaps_present = True
            #     continue
            # else:
            #     data_gaps_present = False
            window = data[window_start:window_end]
            window_mean = window.mean()
            window_std = window.std()            
            
            # 2: z-standardise
            
            I_z = (data[window_end:].dropna() - window_mean) / window_std
            
            peak_flux = np.nanmax(data[window_start:window_end + pd.Timedelta(hours=2)])  # peak flux within window_len + 2 hours
            peaks.append(peak_flux)
            # 3: check next point for 4 sigma deviation. If true, take the last timestamp of bg subtracted flux < 0 as onset
            # or maybe check if few of the next points are all above 4 sigma, so flukes won't trigger the method
            next_points = I_z.iloc[0:5]     # next 5 points
                
            if (next_points > 4).sum() == len(next_points): 
                onset_found = True
                onset_result = window.index[window <= window_mean][-1]
                logging.info(f"Found onset with background {window_start} - {window_end}: {onset_result}")
                onsets.append(onset_result)
                # Write to CSV
                with open(csv_path, "+r", encoding="utf-8") as fp:
                    event_no = len(fp.readlines())
                    fp.write(f"{event_no},{onset_result.date()},{onset_result.time()},{peak_flux}\n")
                    logging.info(f"Results successfully saved to {csv_path}")
                    fp.close()

                # Once an onset has been found, skip 2 hours ahead.
                i += 120 // resample_int
            
            else:
                i += 1

        fig, ax = plt.subplots()

        ax.step(data[date:next_date].index, data[date:next_date])
        if onset_found:
            for time in onsets: 
                ax.axvline(time, color="red")

        ax.set_yscale("log")
        ax.set_ylim(None, 1e2 if max(peaks) < 1e2 else None)
        ax.set_ylabel("Intensity")
        fig.suptitle(f"Onset determination for {date}")
        fig.tight_layout()
        
        fname = plot_dir + os.sep + f"Wind_{date.strftime("%Y%m%d")}"
        fig.savefig(fname, bbox_inches="tight")
        plt.close(fig)
        d += 1

    except requests.exceptions.ReadTimeout:
        logging.info(f"{traceback.format_exc()}")
        if j < MAX_RETRIES:
            j += 1
            sleep(60)
        else:
            logging.info("Download retries exceeded, quitting...")
            break
    except Exception:
        logging.info(f"{traceback.format_exc()}")
        d += 1